# Phase 2: Data Analyst - Exploration & Visualisation

L'Objectif dans cette phase est d'Analyser les données de MovieLens via le SDK et en déduire des tendances et statistiques clés.

In [3]:
# Importation des librairies nécessaires
import pandas as pd
import plotly.express as px # pip install plotly-express
import matplotlib.pyplot as plt # pip install matplotlib
import seaborn as sns # pip install seaborn
from moviesdk import MovieClient, MovieConfig

## Se familiariser avec l'API

In [4]:
# Connexion à l'API via le SDK

config = MovieConfig(movie_base_url="https://movielens-api-rmr7.onrender.com")
client = MovieClient(config=config)

# Vérification que l'API est opérationnelle
client.health_check()

MOVIE_API_BASE_URL in MovieConfig init: https://movielens-api-rmr7.onrender.com


{'message': 'API MovieLens opérationnelle'}

On explore les données comme le ferait un Data Analyst.

In [8]:
# Récupération d’un film pour test
movie = client.get_movie(1)
print(f"Titre : {movie.title}")
print(f"Genres : {movie.genres}")

Titre : Toy Story (1995)
Genres : Adventure|Animation|Children|Comedy|Fantasy


Passons à l'Exploration de la table des évaluations (`ratings`).

In [ ]:
# On récupère une grande portion de la table (à ajuster selon la taille réelle)
ratings_df = client.list_ratings(limit=100, output_format="pandas")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     100 non-null    int64  
 1   movieId    100 non-null    int64  
 2   rating     100 non-null    float64
 3   timestamp  100 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 3.3 KB


In [13]:
# Statistiques sur les utilisateurs
user_counts = ratings_df['userId'].value_counts().reset_index()
user_counts

,userId,count
0,1,100


La méthode `get_analytics()` fournit un moyen **intelligent et efficace** d'estimer la **taille totale** de la table `ratings` sans deviner la limite à l’aveugle. On peut donc :

1. Utiliser `client.get_analytics()` pour récupérer le **nombre total d'évaluations** (`ratings_count`)
2. Utiliser ce total comme `limit` pour appeler `list_ratings`
3. Créer un DataFrame `complete_ratings_df` contenant **toutes** les évaluations

In [23]:
# 1. Récupération des statistiques globales
analytics = client.get_analytics()
print(analytics)

movie_count=9742 rating_count=100836 tag_count=3683 link_count=9742


In [21]:
type(analytics)

moviesdk.schemas.schemas.AnalyticsResponse

In [25]:
# Extraction de la valeur directement
total_ratings = analytics.rating_count
print(f"Nombre total d'évaluations : {total_ratings}")

Nombre total d'évaluations : 100836


In [27]:
# Conversion vers dict
analytics_dict = analytics.model_dump()
total_ratings = analytics_dict["rating_count"]
print(total_ratings)

100836


Si on essaie le code ci-dessous, on obtiendra une erreur 422 et c'est tout à fait mormal.

In [30]:
# 2. Téléchargement complet de la table ratings
complete_ratings_df = client.list_ratings(
    limit=total_ratings,
    output_format="pandas"
)
complete_ratings_df.shape

HTTPStatusError: Client error '422 Unprocessable Entity' for url 'https://movielens-api-rmr7.onrender.com/ratings?skip=0&limit=100836'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/422

L'erreur signifie que l'API **refuse de traiter ta requête** parce que le paramètre `limit=100836` est trop élevé pour elle. En général, les APIs REST ont une **limite maximale de résultats** que tu peux demander en une seule fois — souvent autour de 100, 500, voire 1000. Cela permet non seulement de préserver les performances de l'API et d'éviter des surcharges côté serveur, mais aussi d'assurer un contrôle sur les accès aux données. C'est là l'un des grands avantages de la mise en place d'APIs : elles permettent de **partager des données de manière sécurisée** et d’**encadrer l’utilisation des ressources**.

Ces limites peuvent aussi être mises en place pour des raisons **commerciales**, notamment si l'API est utilisée pour vendre ou monétiser des données. En fixant des limites sur le nombre de résultats ou le nombre d'appels, l'entreprise qui fournit l'API peut contrôler l'accès aux données et éviter des abus tout en gérant ses coûts opérationnels.

Cela permet donc de voir les limites de l'API non seulement sous un angle technique et sécuritaire, mais aussi stratégique et commercial.

On cherchera un moyen de récupérer tous les utilisateurs.

In [14]:
user_counts.columns = ['user_id', 'num_ratings']
user_counts

,user_id,num_ratings
0,1,100


Le Data Analyst discute avec le Développeur de l'API et du SDK et comprend que le SDK moviesdk n’expose pas de méthode explicite pour interroger la table users, ni get_user, ni list_users.

A la lecture de la documention de l'API via Swagger, il va essayer l'option de reconstitution des utilisateurs depuis les ratings.

In [17]:
# Liste des user_id uniques
user_ids = ratings_df['userId'].unique()
print(f"Nombre d'utilisateurs uniques : {len(user_ids)}")
print(f"Exemple d'identifiants utilisateurs : {user_ids[:5]}")

Nombre d'utilisateurs uniques : 1
Exemple d'identifiants utilisateurs : [1]
